In [14]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from street_graph import (create_graph, add_places_to_graph, calculate_population, summarize_traffic_data,
                   assign_routes_to_population, calculate_population_loads, update_weights, plot_heatmap, add_population_column_to_houses )

# --- Load and preprocess data ---
house_path = "Векторные данные/Дома_исходные.shp"
bus_path = "Векторные данные/Остановки_ОТ.shp"
files = {"Streets_исходные": "Векторные данные/Streets_исходные.shp"}

houses = gpd.read_file(house_path).to_crs(epsg=4326)

houses["Apartments"] = houses["Apartments"].fillna(0) 
buses = gpd.read_file(bus_path).to_crs(epsg=4326)
streets = gpd.GeoDataFrame(pd.concat([
    gpd.read_file(path).to_crs(epsg=4326).query("Foot == 1")
    for path in files.values()
], ignore_index=True)).loc[lambda df: df.geometry.type == 'LineString']

point = gpd.GeoDataFrame(geometry=[Point(37.495, 55.555)], crs="EPSG:4326").to_crs(epsg=3857)
radius = 1500

houses = houses.to_crs(epsg=3857)
buses = buses.to_crs(epsg=3857)
streets = streets.to_crs(epsg=3857)

houses = houses[houses.geometry.distance(point.geometry.iloc[0]) <= radius]
buses = buses[buses.geometry.distance(point.geometry.iloc[0]) <= radius]
streets = streets[streets.geometry.distance(point.geometry.iloc[0]) <= radius]

houses = houses.to_crs(epsg=4326)
buses = buses.to_crs(epsg=4326)
streets = streets.to_crs(epsg=4326)

# --- Calculate population in each house ---
houses = add_population_column_to_houses(houses)  # Добавление столбца 'Total_People'

# --- Create graph and add places ---
G, nodes = create_graph(streets)
node_coords = np.array(nodes)
tree = cKDTree(node_coords)

add_places_to_graph(houses, G, tree, node_coords, 'house')
add_places_to_graph(buses, G, tree, node_coords, 'bus_stop')

# --- Assign routes and calculate loads based on population ---
route_distribution = assign_routes_to_population(G, houses, buses, tree, node_coords)
edge_loads = calculate_population_loads(G, route_distribution)

# --- Update weights based on loads and visualize heatmap ---
update_weights(G, edge_loads)

summary = summarize_traffic_data(G, edge_loads, route_distribution, buses)
print(summary)

# plot_heatmap(G, edge_loads)



Calculating loads: 100%|██████████| 4152/4152 [00:00<00:00, 14429.98it/s]

{'num_bus_stops': 24, 'total_people': 2434.9439999999972, 'overloaded_edges_count': 0, 'longest_overloaded_edge_length': 0, 'sytem_score': 1.0}


In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import networkx as nx
from collections import deque
import random

# class QNetwork(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(QNetwork, self).__init__()
#         # Преобразование размерности входа
#         self.fc0 = nn.Linear(input_size, 2553)
#         self.fc1 = nn.Linear(2553, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = F.relu(self.fc0(x))  # Преобразуем размерность
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# Define the Q-Network
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 2869)
        self.fc2 = nn.Linear(2869, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


import numpy as np
import networkx as nx
import random

class GraphEnvironment:
    def __init__(self, graph, start_node, target_node, max_people=800):
        self.graph = graph
        self.start_node = start_node
        self.target_node = target_node
        self.max_people = max_people  # Порог нагруженности для вознаграждения
        self.current_node = start_node

    def reset(self):
        self.current_node = self.start_node
        return self.get_state()
    
    def get_state(self):
        # Возвращаем one-hot представление текущего узла
        state = torch.zeros(len(self.graph.nodes))
        state[list(self.graph.nodes).index(self.current_node)] = 1
        return state

    def step(self, action):
        neighbors = list(self.graph.neighbors(self.current_node))
        if action >= len(neighbors):
            raise ValueError(f"Некорректное действие: {action}. Доступные действия: 0-{len(neighbors) - 1}")
        
        next_node = neighbors[action]
        self.current_node = next_node
        done = self.current_node == self.target_node
        reward = self.calculate_reward(next_node, done)
        
        # Возвращаем one-hot представление состояния
        next_state = self.get_state()  # Убедитесь, что get_state() возвращает корректное one-hot представление
        return next_state, reward, done


    def calculate_reward(self, next_node, done):
        # Получаем длину пути и нагрузку, если ребро существует
        if self.graph.has_edge(self.current_node, next_node):
            path_length = self.graph[self.current_node][next_node].get('weight', 1)
            load = self.graph.nodes[next_node].get('total_people', 0)
        else:
            # Обработка отсутствующего ребра
            print(f"Ребро от {self.current_node} к {next_node} отсутствует.")
            path_length = 1  # Можете задать значение по умолчанию
            load = 0         # Задайте нагрузку по умолчанию, если необходимо

        # Политики вознаграждения
        if path_length <= 10:
            reward = 10 - path_length  # чем короче путь, тем больше награда
        else:
            reward = -path_length

        if load >= 800:
            reward -= 5  # уменьшаем награду за нагруженный путь
        else:
            reward += 5  # увеличиваем награду за менее нагруженный путь

        # Учитываем, если агент достиг цели
        if done:
            reward += 100  # вознаграждение за достижение цели

        return reward


    def action_space(self):
        # Возвращаем количество доступных действий из текущего узла
        return len(list(self.graph.neighbors(self.current_node)))

# Agent with Q-learning
class Agent:
    def __init__(self, state_size, action_size, lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.1):
        self.state_size = state_size
        self.action_size = action_size
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.gamma = gamma
        
        self.q_network = QNetwork(state_size, action_size)
        self.target_network = QNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
        self.memory = deque(maxlen=10000)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.q_network(state)
        return np.argmax(q_values.cpu().data.numpy())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                with torch.no_grad():
                    # Извлекаем тензор из списка
                    if isinstance(next_state, list):
                        next_state_tensor = next_state[0]  # Берем первый элемент списка
                    elif isinstance(next_state, torch.Tensor):
                        next_state_tensor = next_state
                    else:
                        raise ValueError("Unknown type for next_state")
                    
                    # Приводим next_state к нужной размерности
                    next_state_tensor = next_state_tensor.view(1, -1)

                    # Получаем Q-значения для next_state
                    next_state_q_values = self.target_network(next_state_tensor)
                    target += self.gamma * torch.max(next_state_q_values).item()

            # Преобразуем state в тензор аналогичным образом
            if isinstance(state, list):
                state_tensor = state[0]  # Берем первый элемент списка
            elif isinstance(state, torch.Tensor):
                state_tensor = state
            else:
                raise ValueError("Unknown type for state")

            # Приводим state к нужной размерности
            state_tensor = state_tensor.view(1, -1)

            # Получаем предсказание Q-значений для состояния
            target_f = self.q_network(state_tensor)
            target_f[0][action] = target

            # Оптимизация сети
            self.optimizer.zero_grad()
            loss = self.loss_fn(target_f, self.q_network(state_tensor))
            loss.backward()
            self.optimizer.step()


    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

def train_agent(agent, environment, episodes=500, batch_size=32):
    for e in range(episodes):
        state = environment.reset()
        for time in range(200):  # Ограничение количества шагов в эпизоде
            # Получаем доступное количество действий (соседей) для текущего узла
            action_size = environment.action_space()
            
            # Подаем это значение в метод `act` для выбора корректного действия
            action = agent.act([state]) % action_size  # Обеспечиваем, что action < action_size
            
            next_state, reward, done = environment.step(action)
            agent.remember([state], action, reward, [next_state], done)
            
            state = next_state
            if done:
                print(f"Episode {e+1}/{episodes} finished after {time+1} timesteps")
                break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)



# # Graph-based environment setup example
# G = nx.DiGraph()
# G.add_edge(0, 1, weight=1, load=700)
# G.add_edge(1, 2, weight=2, load=850)  # Example of a highly loaded edge
# G.add_edge(2, 3, weight=1, load=600)
# G.add_edge(3, 4, weight=1, load=500)
# G.add_edge(4, 0, weight=2, load=750)

# Initialize environment, agent, and train
state_size = 1  # Node-based state representation
action_size = 2  # Number of neighbors for each node
start_node = nodes[0]
end_node = nodes[-1]
environment = GraphEnvironment(G, start_node=start_node, target_node=end_node)
agent = Agent(state_size=len(nodes), action_size=environment.action_space())

# Тренируем агента
train_agent(agent, environment, episodes=500, batch_size=32)


Ребро от (37.497338608489436, 55.5484909701677) к (37.497338608489436, 55.5484909701677) отсутствует.
Ребро от (37.497476320222475, 55.54843186824961) к (37.497476320222475, 55.54843186824961) отсутствует.
Ребро от (37.497338608489436, 55.5484909701677) к (37.497338608489436, 55.5484909701677) отсутствует.
Ребро от (37.4973301643322, 55.54861938818431) к (37.4973301643322, 55.54861938818431) отсутствует.
Ребро от (37.497338608489436, 55.5484909701677) к (37.497338608489436, 55.5484909701677) отсутствует.
Ребро от (37.49722407329069, 55.548528931541995) к (37.49722407329069, 55.548528931541995) отсутствует.
Ребро от (37.49720359170221, 55.548429428959594) к (37.49720359170221, 55.548429428959594) отсутствует.
Ребро от (37.49722407329069, 55.548528931541995) к (37.49722407329069, 55.548528931541995) отсутствует.
Ребро от (37.49669801986032, 55.548705931682754) к (37.49669801986032, 55.548705931682754) отсутствует.
Ребро от (37.496491676839554, 55.54876198410206) к (37.496491676839554, 55

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2869 and 2553x2869)